In [1]:
def str2columned_list(string, num):
    s = string
    first_column = s[:s.find(" ")  ] 
        
    second = s[s.find(" ") + 1:] 
    second_column = second[:second.find(' ')]
    
    third = second[second.find(" ") + 1:]
    third_column = third[:third.find(' ')]
    
    fourth = third[third.find(" ") + 1:] 
    if num ==6:
        fourth_column = fourth[:fourth.find(' ')]
        fifth = fourth[fourth.find(" ") + 1:] 
        fifth_column = fifth[:fifth.find(' ')]
        sixth = fifth[fifth.find(' ')+1:]
    
    if num ==4:
        return [first_column, second_column, third_column, fourth]
    if num == 6: 
        return [first_column, second_column, third_column, fourth_column, fifth_column, sixth]

## imports

In [1]:
import pandas as pd
import os

In [2]:
import re
import copy


## files reading

In [3]:
dir_path = 'factRuEval-2016-master\mix'

In [4]:
file_pathes = []
for top, dirs, files in os.walk(dir_path):
    for nm in files:
        file_pathes.append(os.path.join(top, nm))

In [5]:
files = []
for i in range(0, len(file_pathes)):
    files.append(os.path.basename(file_pathes[i]))
len(files)


1524

## data reading

In [6]:
toks = []
spns = []
txts = []
for file in files:
    if os.path.splitext(file)[1]=='.tokens':
        toks.append(file)
    if os.path.splitext(file)[1]=='.spans':
        spns.append(file)
    if os.path.splitext(file)[1]=='.txt':
        txts.append(file)

In [7]:
# CONTENT

In [10]:
texts_data = []
for file in txts:
    with open(dir_path+'\\'+file, "r", encoding='utf-8') as ins:
        text = ins.read()
        #text = text.replace('\n', ' ') #спорное утверждение, на самом деле
    texts_data.append(text)

In [11]:
spans = []
for file in spns:
    span = []
    for line in open(dir_path+'\\'+file, "r", encoding='utf-8'):
        span.append(line)   
    spans.append(span)

## LOOP

In [12]:
# TAG, START, END

In [14]:
len(texts_data) == len(spans)

True

In [16]:
data = []
for l in range(0, len(texts_data)):
    tags = []
    starts = []
    lens = []
    ends = []
    for i in range(0, len(spans[l])):
        s = spans[l][i]
        parts = s.split(' ') 

        if parts[1]=='name' or parts[1]=='surname' or parts[1]=='nickname' or parts[1]=='loc_name' or parts[1]=='org_name': 

            tags.append(parts[1])
            starts.append(parts[2])
            lens.append(parts[3])
            ends.append(int(parts[2])+int(parts[3]))
            

    for i in range(0, len(tags)):
        if tags[i]=='loc_name':
            tags[i] = 'LOC'
        if tags[i]=='org_name':
            tags[i] = 'ORG'
        if tags[i] == 'name' or tags[i] == 'surname' or tags[i]== 'nickname':
            tags[i] = 'PER'    
            
            
    content = texts_data[l]
    tagged_content = copy.copy(content)
    

    list_content = list(tagged_content)

    for i in range(0, len(list_content)):
        if list_content[i] != ' ':
            list_content[i] = 'O'

    tagged_content = "".join(list_content)

    

    for i in range(0, len(tags)):
        for j in range(int(starts[i]), int(ends[i])):
            if content[j] == ' ':
                list_content[j] = ' '
            else:
                list_content[j] = tags[i]

    tagged_content2 = "".join(list_content)
    
    
    
    tagged_content2 = re.sub(r'PER+', 'P', tagged_content2)
    tagged_content2 = re.sub(r'P+', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'LOC+', 'L', tagged_content2)
    tagged_content2 = re.sub(r'L+', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'O+', 'O', tagged_content2)
    tagged_content2 = re.sub(r'ORG+', 'O', tagged_content2)
    tagged_content2 = re.sub(r'OO+', 'ORG', tagged_content2)
    
    
    
    tagged_content2 = re.sub(r'OPER', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'PERO', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'OLOC', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'LOCO', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'OORG', 'ORG', tagged_content2)
    tagged_content2 = re.sub(r'ORGO', 'ORG', tagged_content2)
    
    
    tagged_content2 = re.sub(r'PER+', 'P', tagged_content2)
    tagged_content2 = re.sub(r'P+', 'PER', tagged_content2)
    tagged_content2 = re.sub(r'LOC+', 'L', tagged_content2)
    tagged_content2 = re.sub(r'L+', 'LOC', tagged_content2)
    tagged_content2 = re.sub(r'O+', 'O', tagged_content2)
    tagged_content2 = re.sub(r'ORG+', 'O', tagged_content2)
    tagged_content2 = re.sub(r'OO+', 'ORG', tagged_content2)
    
    
    
    data.append([tagged_content2, content])

df = pd.DataFrame(data, columns=['tags','texts'])
df.to_csv('FactRuEval_ultimate_mix.csv', index = False)

## check

In [23]:
idx = 60

In [24]:
words = df['texts'][idx].split(' ')
tags = df['tags'][idx].split(' ')

In [25]:
for i in range(len(words)):
    print('word ', words[i])
    print('tag ', tags[i])
    print('\n')

word  Экс-премьера
tag  O


word  Пакистана
tag  LOC


word  Беназир
tag  PER


word  Бхутто
tag  PER


word  вновь
tag  O


word  взяли
tag  O


word  под
tag  O


word  домашний
tag  O


word  арест
tag  O


word  за
tag  O


word  несогласование
tag  O


word  митинга
tag  O


word  против
tag  O


word  введения
tag  O


word  чрезвычайного
tag  O


word  положения
tag  O


word  с
tag  O


word  властями.
tag  O


word  Дом
tag  O


word  Бхутто
tag  PER


word  в
tag  O


word  Лахоре
tag  LOC


word  оцеплен
tag  O


word  полицией.
tag  O


word  Она
tag  O


word  не
tag  O


word  имеет
tag  O


word  права
tag  O


word  покидать
tag  O


word  дом
tag  O


word  в
tag  O


word  течение
tag  O


word  7
tag  O


word  дней,
tag  O


word  она
tag  O


word  не
tag  O


word  сможет
tag  O


word  возглавить
tag  O


word  запланированную
tag  O


word  манифестацию
tag  O


word  против
tag  O


word  чрезвычайного
tag  O


word  положения
tag  O


word  в
tag  O


word  Па

# NOT LOOP

In [90]:
raw_data = []
for line in open(dir_path+ '//'+'book_100.spans', 'r', encoding='utf-8'):
    raw_data.append(line)
raw_data

['22763 loc_name 37 6 143790 1  # 143790 Москвы\n',
 '22764 org_descr 31 5 143789 1  # 143789 мэрии\n',
 '22765 loc_name 47 8 143792 1  # 143792 Тверской\n',
 '22766 loc_descr 56 7 143793 1  # 143793 площади\n',
 '22767 name 313 4 143831 1  # 143831 Юрия\n',
 '22768 surname 318 7 143832 1  # 143832 Лужкова\n',
 '22769 loc_name 306 6 143830 1  # 143830 Москвы\n',
 '22770 loc_name 477 6 143853 1  # 143853 Москве\n',
 '22771 loc_name 531 6 143862 1  # 143862 России\n',
 '22772 org_name 562 10 143868 1  # 143868 Мосгордумы\n',
 '22773 name 610 6 143874 1  # 143874 Бориса\n',
 '22774 surname 617 7 143875 1  # 143875 Громова\n',
 '22775 name 756 4 143899 1  # 143899 Юрия\n',
 '22776 surname 761 7 143900 1  # 143900 Лужкова\n',
 '22777 job 301 4 143829 1  # 143829 мэра\n',
 '22778 job 598 11 143873 1  # 143873 губернатора\n',
 '22779 job 584 25 143872 2  # 143872 143873 подмосковного губернатора\n',
 '68733 geo_adj 584 13 143872 1  # 143872 подмосковного\n',
 '22782 name 968 4 143934 1  # 143

In [91]:
tags = []
starts = []
lens = []
ends = []
for i in range(0, len(raw_data)):
    s = raw_data[i]
    parts = s.split(' ') 
    
    if parts[1]=='name' or parts[1]=='surname' or parts[1]=='nickname' or parts[1]=='loc_name' or parts[1]=='org_name': 

        tags.append(parts[1])
        starts.append(parts[2])
        lens.append(parts[3])
        ends.append(int(parts[2])+int(parts[3]))

In [92]:
for i in range(0, len(tags)):
    if tags[i]=='loc_name':
        tags[i] = 'LOC'
    if tags[i]=='org_name':
        tags[i] = 'ORG'
    if tags[i] == 'name' or tags[i] == 'surname' or tags[i]== 'nickname':
        tags[i] = 'PER'

In [93]:
content = text_list[0]

In [94]:
content

'В понедельник 28 июня у здания мэрии Москвы на Тверской площади состоялась очередная несанкционированная акция протеста « День гнева » , в этот раз направленная , главным образом , против политики московских и подмосковных властей .  Среди требований , выдвигаемых организаторами акции : « немедленная отставка мэра Москвы Юрия Лужкова , расследование итогов его деятельности » , « созыв московского общественного форума для обсуждения путей реформирования основных сфер жизнедеятельности в Москве » , « восстановление прямых выборов глав регионов России » , « роспуск нелегитимной Мосгордумы » , отставка подмосковного губернатора Бориса Громова и др .  Участникам акции предлагалось принести с собой лист бумаги или кусок ткани чёрного цвета , символизирующие « чёрную метку » для Юрия Лужкова .  Начало акции было намечено на 19 часов ; подчёркивалось , что она состоится несмотря на запрет властей .  Освещающие акцию блоггеры сообщили , что автобусы с милицией стали занимать площадь у памятник

In [95]:
tagged_content = copy.copy(content)
tagged_content

'В понедельник 28 июня у здания мэрии Москвы на Тверской площади состоялась очередная несанкционированная акция протеста « День гнева » , в этот раз направленная , главным образом , против политики московских и подмосковных властей .  Среди требований , выдвигаемых организаторами акции : « немедленная отставка мэра Москвы Юрия Лужкова , расследование итогов его деятельности » , « созыв московского общественного форума для обсуждения путей реформирования основных сфер жизнедеятельности в Москве » , « восстановление прямых выборов глав регионов России » , « роспуск нелегитимной Мосгордумы » , отставка подмосковного губернатора Бориса Громова и др .  Участникам акции предлагалось принести с собой лист бумаги или кусок ткани чёрного цвета , символизирующие « чёрную метку » для Юрия Лужкова .  Начало акции было намечено на 19 часов ; подчёркивалось , что она состоится несмотря на запрет властей .  Освещающие акцию блоггеры сообщили , что автобусы с милицией стали занимать площадь у памятник

In [96]:
list_content = list(tagged_content)

for i in range(0, len(list_content)):
    if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
        list_content[i] = 'O'
        
tagged_content = "".join(list_content)

In [97]:
#list_content = list(tagged_content[0])

for i in range(0, len(tags)):
    for j in range(int(starts[i]), int(ends[i])):
        if content[j] == ' ':
            list_content[j] = ' '
        else:
            list_content[j] = tags[i]
        
tagged_content2 = "".join(list_content)

In [98]:
import re
tagged_content2 = re.sub(r'PER+', 'P', tagged_content2)
tagged_content2 = re.sub(r'P+', 'PER', tagged_content2)
tagged_content2 = re.sub(r'LOC+', 'L', tagged_content2)
tagged_content2 = re.sub(r'L+', 'LOC', tagged_content2)
tagged_content2 = re.sub(r'O+', 'O', tagged_content2)
tagged_content2 = re.sub(r'ORG+', 'O', tagged_content2)
tagged_content2 = re.sub(r'OO+', 'ORG', tagged_content2)
tagged_content2

'O O O O O O O LOC O LOC O O O O O O O O O O O O O O O O O O O O O O O O O O  O O O O O O O O O OLOC LOCOPER PEROPER PERO O O O O O O O O O O O O O O O O O O O OLOCO O O O O O O O OLOC LOCO O O O O O ORG ORG O O O O OPEROPER PERO O O O O O  O O O O O O O O O O O O O O OPERO PER PERO O O O O O O  O O O O O O O O O O O O O O O O O O  O O O O O O O O O OPER OPER PERO O O O O O O O O O O O O O O O O  O O O O O O O O O O O O O O O O O O O O O O O O O O O O O  O O O O O O O O O OPER O O O O  O O O O O O O O OLOC LOCO O ORG ORG ORG O ORG OPEROPER PERO O O OPER PER PER PERO O O ORG ORG PEROPER PER PERO O O O O O O ORG O  ORG ORG ORG ORG O ORG O ORG O O ORG ORG ORG O ORG O O ORG O ORG ORG O O O ORG LOC LOC O O ORG ORG O ORG O ORG ORG ORG O ORG O ORG O O O O O O O O O O '

In [103]:
tagged_content2 = re.sub(r'OPER', 'PER', tagged_content2)
tagged_content2 = re.sub(r'PERO', 'PER', tagged_content2)
tagged_content2 = re.sub(r'OLOC', 'LOC', tagged_content2)
tagged_content2 = re.sub(r'LOCO', 'LOC', tagged_content2)
tagged_content2 = re.sub(r'OORG', 'ORG', tagged_content2)
tagged_content2 = re.sub(r'ORGO', 'ORG', tagged_content2)

In [100]:
words = content.split(' ')
tags = tagged_content2.split(' ')

In [101]:
len(tags)==len(words)

True

In [104]:
for i in range(len(words)):
    print('word ', words[i])
    print('tag ', tags[i])
    print('\n')

word  В
tag  O


word  понедельник
tag  O


word  28
tag  O


word  июня
tag  O


word  у
tag  O


word  здания
tag  O


word  мэрии
tag  O


word  Москвы
tag  LOC


word  на
tag  O


word  Тверской
tag  LOC


word  площади
tag  O


word  состоялась
tag  O


word  очередная
tag  O


word  несанкционированная
tag  O


word  акция
tag  O


word  протеста
tag  O


word  «
tag  O


word  День
tag  O


word  гнева
tag  O


word  »
tag  O


word  ,
tag  O


word  в
tag  O


word  этот
tag  O


word  раз
tag  O


word  направленная
tag  O


word  ,
tag  O


word  главным
tag  O


word  образом
tag  O


word  ,
tag  O


word  против
tag  O


word  политики
tag  O


word  московских
tag  O


word  и
tag  O


word  подмосковных
tag  O


word  властей
tag  O


word  .
tag  O


word  
tag  


word  Среди
tag  O


word  требований
tag  O


word  ,
tag  O


word  выдвигаемых
tag  O


word  организаторами
tag  O


word  акции
tag  O


word  :
tag  O


word  «
tag  O


word  немедленная
tag  O


word 